In [7]:
import praw

CLIENT_ID = "CLIENT_ID"
SECRET = "SECRET"
USER_AGENT = "USER_AGENT"

In [8]:
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=SECRET,
    user_agent=USER_AGENT,
)

print(reddit.read_only)

True


In [9]:
from datetime import datetime
import pandas as pd

with open('reddit_links.txt', 'r') as file:
    for index, line in enumerate(file):
        if index != 0:
            continue
        line = line.strip()
        temp = line.split('/')
        submission_id = temp[6].strip()
        name = temp[7]
        comments_data = set()
        submission = reddit.submission(id=submission_id)
        cutoff = datetime(2023, 4, 1)

        submission.comments.replace_more(limit=None)

        comments_data = []
        for comment in submission.comments.list():
            if comment.author:  # Check if the author exists
                author = comment.author.name
            else:
                author = "[deleted]"

            comments_data.append({
                'Comment': comment.body,
                'Author': author,
                'Created UTC': pd.to_datetime(comment.created_utc, unit='s')
            })
        
        df_comments = pd.DataFrame(comments_data)
        df_comments['Created UTC'] = df_comments['Created UTC'].dt.strftime('%Y-%m-%d %H:%M:%S')

        df_filtered = df_comments[pd.to_datetime(df_comments['Created UTC']) <= cutoff]

        df_filtered.to_csv(f'{name}.csv', index=False, encoding='utf-8-sig')

TooManyRequests: received 429 HTTP response